## BTC 1m test cronjob

In [12]:
import datetime as dt
import time
# import my_functions
import importlib
importlib.reload(my_functions)
from my_functions import *

In [2]:
from myConfig import binanceAPI

In [17]:
binanceTest().fetch_balance()

{'info': {'makerCommission': '0',
  'takerCommission': '0',
  'buyerCommission': '0',
  'sellerCommission': '0',
  'commissionRates': {'maker': '0.00000000',
   'taker': '0.00000000',
   'buyer': '0.00000000',
   'seller': '0.00000000'},
  'canTrade': True,
  'canWithdraw': False,
  'canDeposit': False,
  'brokered': False,
  'requireSelfTradePrevention': False,
  'updateTime': '1683266115366',
  'accountType': 'SPOT',
  'balances': [{'asset': 'BNB',
    'free': '1000.00000000',
    'locked': '0.00000000'},
   {'asset': 'BTC', 'free': '0.00000000', 'locked': '0.00000000'},
   {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
   {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
   {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
   {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
   {'asset': 'USDT', 'free': '38902.43345504', 'locked': '0.00000000'},
   {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.0000000

In [3]:
binanceAPI['apiKey']

'5Q5zzfUtxW7HDie0G5Gul3yTvaA5DEQVzbbqui2rWSdd2C1zfbJ5g8BLQW50yNQ7'

In [2]:
pd.set_option('display.max_rows', 200) 

In [3]:
# Define the time periods for the moving averages and the Bollinger Bands
length1 = 5
length2 = 26
bbPeriod = 32
bbStdDev = 1

# Define RSI configuration
rsiLength = 18

# Define stop loss in percentage
stop_loss = 5.7

# Define trading parameters
symbol = 'BTC/USDT'
usdt_amount = 50
timeframe = '4h'
rsi_tf = '1d'

tradesfile = "btcTrades.csv"
logfile = "btc.csv"

In [5]:
def binance():
    exchange = ccxt.binance({
        'apiKey': binanceAPI['apiKey'],
        'secret': binanceAPI['secretKey'],
        'enableRateLimit': True,
        'rateLimit': 10000,
        'options': {
            'recvWindow': 20000,  # replace with your desired recv_window value
            'test': False,  # use testnet (sandbox) environment
            'adjustForTimeDifference': True,
        }
    })
    exchange.set_sandbox_mode(enable=False)
    return exchange


In [6]:
getdata(symbol, timeframe, limit=1500,
         length1=length1,
         length2=length2,
         bbPeriod=bbPeriod,
         bbStdDev=bbStdDev,
         rsi_tf=rsi_tf,
         rsiLength=rsiLength,
         )

InvalidNonce: binance {"code":-1021,"msg":"Timestamp for this request was 1000ms ahead of the server's time."}

In [8]:
exchange.set_sandbox_mode(enable=False)

In [9]:
    ohlcv = exchange.fetch_ohlcv(symbol, "1d",limit=500)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume']).iloc[:-1]
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df = df[['open', 'high', 'low', 'close', 'volume']]
    df


InvalidNonce: binance {"code":-1021,"msg":"Timestamp for this request was 1000ms ahead of the server's time."}

In [4]:
in_pos = in_pos("BTC")
in_position = in_pos[0]
asset = in_pos[2]

In [5]:
size = calculate_order_size(symbol, usdt_amount)
qty = asset

# cronjob code
 
try:
    df = getdata(symbol, timeframe, limit=100,
         length1=length1,
         length2=length2,
         bbPeriod=bbPeriod,
         bbStdDev=bbStdDev,
         rsi_tf=rsi_tf,
         rsiLength=rsiLength,
         )
    print(df.iloc[-1:])
    # Check for buy and sell signals
    signal = df['buy'][-1]
    print(dt.datetime.now())
    if signal == True and not in_position:
        # Place buy order
        buyId = place_buy_order(symbol, size)
        in_position = True
        buyprice = float(buyId['info']['fills'][0]['price'])
        qty = float(buyId['info']['origQty'])
        buycsv(df, buyprice, tradesfile)
        print(f'Buy order placed for {symbol} at {buyprice}')

    elif df['sell'][-1] == True and in_position:
        # Place sell order
        sellId = place_sell_order(symbol, qty)
        in_position = False
        sellprice = float(sellId['info']['fills'][0]['price'])
        buyprice = read_buyprice()
        profit = ((sellprice - buyprice) / buyprice- 0.002) * 100
        sellcsv(df, buyprice, sellprice, tradesfile)
        print(f'Sell order placed for {symbol} at {sellprice}, Profit: {profit:.2f}%')

    # Check for stop loss
    elif in_position and (df['close'][-1] / buyprice - 1) * 100 < -stop_loss/100:
        # Place sell order
        sellId = place_sell_order(symbol, qty)
        in_position = False
        sellprice = float(sellId['info']['fills'][0]['price'])
        buyprice = read_buyprice("btcTrades")
        profit = ((buyprice - sellprice) / buyprice- 0.002) * 100
        sellcsv(df, buyprice, sellprice, tradesfile)
        print(f'Sell order placed for {symbol} at {sellprice}, Profit: {profit:.2f}%')

# write the last row to the CSV file    
#         with open('btc.csv', 'a', newline='') as f:
#             df.iloc[-1:].to_csv(f, header=f.tell())   

    csvlog(df, logfile)

except Exception as e:
    print(e)

# sec_of_execution = dt.datetime.now().second+(dt.datetime.now().microsecond/1000000)
# time.sleep(60-sec_of_execution+5)

                         open      high       low     close    volume  \
timestamp                                                               
2023-05-05 10:49:00  29107.88  29115.86  29107.87  29115.86  2.741271   

                          sma1          sma2      bb_upper        RSI    buy  \
timestamp                                                                      
2023-05-05 10:49:00  29099.848  29102.481538  29114.837969  58.485166  False   

                     sell  
timestamp                  
2023-05-05 10:49:00  True  
2023-05-05 15:50:07.851746
